# Reto AD2023

## Descripción 

El reto consiste en desarrollar un sistema multiagente para resolver una tarea cooperativa en un entorno 20x20 dinámicamente cambiante. El entorno del sistema multiagente es un mundo similar a una cuadrícula donde los agentes pueden moverse de su posición a una celda vecina si ya no hay ningún agente en esa ranura. En este entorno, la comida puede aparecer en cualquier celda menos en una. La celda especial, en la que no puede aparecer comida, se considera un depósito donde los agentes pueden traer y almacenar su comida. Un agente puede sólo puede saber si hay comida en una celda, si está visitándola. Inicialmente, la comida se coloca en algunas celdas aleatorias. Durante la ejecución, puede aparecer comida adicional dinámicamente en celdas seleccionadas al azar, excepto en la celda del depósito. Los agentes pueden tener/desempeñar diferentes roles (como explorador o recolector), comunicarse y cooperar para encontrar y recolectar alimentos de manera eficiente y efectiva

## Puntos a considerar

- Inicialmente no hay comida en el entorno.
- La semilla para generación de números aleatorios será 12345.
- El depósito será generado al azar.
- Cada 5 segundos se colocará una unidad de comida en algunas celdas.
- La cantidad de celdas en las que colocará una unidad comida será definida al azar (entre 2 y 5 celdas).
- Se colocará un total de 47 unidades de comida.
- Número total de pasos (steps): 1500.
- La cantidad total de alimentos que se puede almacenar en el depósito es infinito.
- Hay un total de 5 agentes.
- Cuando una unidad de comida es encontrado por un explorador o por un agente que ya lleva la comida, la posición de la comida se marca y se comunica a otros agentes.
- Cuando un recolector encuentra una unidad comida, lo carga (gráficamente deberá cambia su forma para indicar que lleva comida). La capacidad máxima de comida que puede llevar un agentes es UNA unidad de comida.
- Inicialmente, los agentes no son informados sobre la posición del depósito, pero una vez que lo encuentran, todos saben dónde está.

____

### Imports

In [ ]:
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random 

In [ ]:
SEED = 12345
MAX_STEPS = 1500
MAX_AGENTS = 5
MAX_FOOD = 47

### Agent(s)

In [ ]:
# CAMBIAR QUE PUEDA SER MAS DE UNA CELDA QUE TENGA COMIDA
class RobotAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        # Attribute to set if the agent is carrying food or not
        self.is_carrying = False
        # Attribute to store the position of the deposit
        self.deposit_cell = None
        # Attribute to store the position of where the food is
        self.food_cell = None
    
    def step(self):
        # Check if a cell with food has been found
        self.get_food_cell()

        # Obtain the current position of the cell
        x, y = self.pos

        # Variable to store the food of the cell
        food_in_cell = self.model.get_food_in_cell(x, y)

        # If the current cell has food and the model doesn't have set a cell with food, then set the current cell and pick food
        if food_in_cell > 0 and self.model.get_food_cell() == None:
            self.set_food_cell(x, y)
            # Pick the food of the current cell
            self.pick_food(x, y)

        # Else, if the current cell has food and the model has a cell with food set, then pick food
        elif food_in_cell > 0:
            self.pick_food(x, y)
            # If the cell where the food was has now become empty, then the food cell has no more food and it must be reset
            if self.model.get_food_in_cell(x, y) == 0:
                self.reset_food_cell()
        
        self.move()

    # Move the agent
    def move(self):
        # If a cell with food has been found
        if self.food_cell != None:
            if self.is_carrying and self.deposit_cell != None:
                pass
            # If the deposit hasn't been found, then move random
            elif self.is_carrying:
                # Move to a cell which doesn't have an agent
                possible_moves = [cell for cell in self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False) if self.model.grid.is_cell_empty(cell)]

                # If there are possible moves then move to a random cell
                if possible_moves:
                    new_position = random.choice(possible_moves)
                    self.model.grid.move_agent(self, new_position)
            # If the agent is not carrying then move to the food cell
            else: 
                # Implement logic to move to the cell that has food
                pass
            
        # If a cell with food hasn't been found, then move random
        else:
            # Move to a cell which doesn't have an agent
            possible_moves = [cell for cell in self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False) if self.model.grid.is_cell_empty(cell)]

            # If there are possible moves then move to a random cell
            if possible_moves:
                new_position = random.choice(possible_moves)
                self.model.grid.move_agent(self, new_position)

    # Method to check if the agent is carrying, if not then pick food from the cell
    def pick_food():
        pass

     # Method to check if the agent is carrying and it has arrived at the deposit, then place the food in the deposit
    def place_food():
        pass

    # Method to check if a cell with food has been found
    def get_food_cell(self):
        # If a cell with food has been found, then set the attribute of the agent with the position of the cell with food
        if self.model.get_food_cell() != None:
            self.food_cell = self.model.get_food_cell()
    
    # Method to set the cell with food
    def set_food_cell(self, x, y):
        self.model.set_food_cell(x, y)
        self.food_cell = self.model.get_food_cell()

    # Method to reset the cell with food
    def reset_food_cell(self):
        self.model.set_food_cell(None, None)
        self.food_cell = None

### Model

In [ ]:
class FoodModel(Model):
    def __init__(self, width, height, num_agents, max_food):
        self.random.seed(SEED)
        self.num_agents = num_agents
        self.max_food = max_food
        self.grid = SingleGrid(width, height, torus=True)
        self.DataCollector = DataCollector(
            model_reporters={"Food": "food"},
            agent_reporters={"Position": "pos"}
        )
        self.schedule = SimultaneousActivation(self)
        self.deposit_cell = (random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1))
        self.food_generated = 0

        def generate_agents(self, num_agents):
            for i in range(num_agents):
                agent = 
